- Do operations
- Do forward prop
- Do backward prop

In [2]:
import math
import random
import numpy as np
import matplotlib.pyplot as plt


In [3]:
class Neuron:
    
    def __init__(self, nin):
        self.w = [Value(random.uniform(-1, 1)) for _ in range(nin)]
        self.b = Value(random.uniform(-1, 1))
    
    def __call__(self, x):
        act = sum((wi*xi for wi,xi in zip(self.w, x)), self.b)
        out = act.tanh()
        return out
    
    def parameters(self):
        return self.w + [self.b]
    
class Layer:
    
    def __init__(self, nin, nout):
        self.neurons = [Neuron(nin) for _ in range(nout)]
        
    def __call__(self, x):
        outs = [n(x) for n in self.neurons]
        return outs[0] if len(outs) == 1 else outs
    
    def parameters(self):
        return [p for neuron in self.neurons for p in neuron.parameters()]

class MLP:

    def __init__(self, nin, nouts):
        sz = [nin] + nouts
        self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]

    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]
        

In [4]:
class Value:
    
    def __init__(self, data, _children=(), _op='', label=''):
        self.label = label
        self.data = data
        self._prev = set(_children)
        self._op = _op
        self.grad = 0.0
        self._backward = lambda: None

    def __repr__(self):
        return f"Value(data={self.data})"
    
    def __add__(self, other):
        out = Value(self.data+other.data, (self, other), '+')
        
        def _backward(self, other):
            self.grad += out.grad 
            other.grad += out.grad
        out._backward = _backward    
        return out

    def __mul__(self, other):
        out = Value(self.data*other.data, (self, other), '*')
        
        def _backward(self, other):
            self.grad += out.grad*other.data
            other.grad += out.grad*self.data

        out._backward = _backward  
        return out
    
    def tanh(self):
        x = Value((math.exp(2*self.data)-1)/(math.exp(2*self.data) +1), (self, ), 'tanh')
        
        def _backward(self):
            self.grad += (1-x.data**2)*x.grad
        x._backward = _backward
            
        return x
    
    def backward(self):
        
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
            for child in v._prev:
                build_topo(child)
            topo.append(v)
        build_topo(self)    
        
        self.grad = 1.0
        for node in reversed(topo):
            node._backward      
            


In [5]:
a = Value(4.0, label='a')
b = Value(-7.0, label='b')
c = Value(0.05); c.label='c'

d = a+b; d.label='d'
e = d*c; e.label='e'
m = e.tanh()
print(m.data)


-0.14888503362331798


In [7]:
print(b.grad)

0.0
